In [1]:
#Import Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold,train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

from sklearn.model_selection import StratifiedKFold,KFold,GridSearchCV
from sklearn.feature_selection import RFECV,RFE,f_regression,mutual_info_regression,VarianceThreshold,SelectFromModel,mutual_info_regression
from sklearn.feature_extraction.image import grid_to_graph
from sklearn import feature_selection
from sklearn.cluster import FeatureAgglomeration
from sklearn.linear_model import BayesianRidge


from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from math import sqrt
from sklearn.metrics import r2_score

import eli5
from eli5.sklearn import PermutationImportance

import shutil
import tempfile
from scipy import linalg, ndimage
from joblib import Memory as mem

C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [91]:
# https://machinelearningmastery.com/feature-selection-in-python-with-scikit-learn/
# https://www.datacamp.com/community/tutorials/feature-selection-python
# https://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/
# https://medium.com/@feraguilari/multiple-regression-and-recursive-feature-elimination-rfe-34af0c6ae51b
# https://topepo.github.io/caret/recursive-feature-elimination.html
# https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15
# https://medium.com/@mehulved1503/feature-selection-and-feature-extraction-in-machine-learning-an-overview-57891c595e96
# https://bookdown.org/max/FES/classes-of-feature-selection-methodologies.html
# https://topepo.github.io/caret/recursive-feature-elimination.html

df = pd.read_csv('2ex1.csv',header=0)
df.head()
df1 = pd.read_csv('1ex1.csv',header=0)
#df2= pd.read_csv('1ex2.csv',header=0)
df3= pd.read_csv('1ex3.csv',header=0)
#df4= pd.read_csv('1ex4.csv',header=0)
df5= pd.read_csv('1ex5.csv',header=0)
#df6= pd.read_csv('1ex6.csv',header=0)
df7= pd.read_csv('1ex7.csv',header=0)
#df8= pd.read_csv('1ex8.csv',header=0)
df9= pd.read_csv('1ex9.csv',header=0)
df10= pd.read_csv('2ex10.csv',header=0)
df11= pd.read_csv('2ex11.csv',header=0)
df12= pd.read_csv('2ex12.csv',header=0)
df13= pd.read_csv('2ex13.csv',header=0)
df14= pd.read_csv('2ex14.csv',header=0)
df15= pd.read_csv('2ex15.csv',header=0)

#All features
#X = df[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','DS_DSDT','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground isColliding','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_Slew_Force','DS_Bkt_Force','DS_Dipper_Force','DS_BoomR_Force','DS_BoomL_Force','DS_CylinderBoomLiftL','DS_CylinderBoomLiftR','DS_CylinderDipperLift','DS_CylinderBucketLift','DS_Slew','DS_Bucket_Angle','DS_Dipper_Angle','DS_Boom_Angle','DS_Slew_Angle','DS_BoomCylinderPressureA','DS_BoomCylinderPressureB','DS_DipperCylinderPressureA','DS_DipperCylinderPressureB','DS_BucketCylinderPressureA','DS_BucketCylinderPressureB','DS_Boom_zdd','DS_Boom_ydd','DS_Boom_xdd','DS_Dipper_zdd','DS_Dipper_ydd','DS_Dipper_xdd','DS_Bkt_zdd','DS_Bkt_ydd','DS_Bkt_xdd','DS_Bktwx','DS_Bktwy','DS_Bktwz','DS_Dipperwx','DS_Dipperwy','DS_Dipperwz','DS_Boomwx','DS_Boomwy','DS_Boomwz','DS_Upperwx','DS_Upperwy','DS_Upperwz']]
#y = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]

#Removed correlated
#Data import Bulk correlations

X = df.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X)
X = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y)
y = pd.DataFrame(scaler_y)

X2 = df2.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y2 = df2[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X2)
X2 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y2)
y2 = pd.DataFrame(scaler_y)

X3 = df3.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y3 = df3[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X3)
X3 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y3)
y3 = pd.DataFrame(scaler_y)

X4 = df4.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y4 = df4[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X4)
X4 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y4)
y4 = pd.DataFrame(scaler_y)

X5 = df5.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y5 = df5[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X5)
X5 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y5)
y5 = pd.DataFrame(scaler_y)

X6 = df6.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y6 = df6[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X6)
X6 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y6)
y6 = pd.DataFrame(scaler_y)

X7 = df7.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y7 = df7[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X7)
X7 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y7)
y7 = pd.DataFrame(scaler_y)

X8 = df8.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','MassHopper Mass','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue'],axis=1)
y8 = df8[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X8)
X8 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y8)
y8 = pd.DataFrame(scaler_y)

X9 = df9.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','soilTransferSensor MassTot','Collisionsensor_Ground isColliding','DS_DI_SlewPos','DS_DI_SlewNeg','Uppercarriage xd','Uppercarriage yd','Uppercarriage zd','MainBoom xd','MainBoom yd','MainBoom zd','DipperArm xd','DipperArm yd','DipperArm zd','Bucket xd','Bucket yd','Bucket zd','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','SoilTransferSensor2 SoilTot','soilTransferSensor SoilTot'],axis=1)
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','soilTransferSensor MassTot','Collisionsensor_Ground isColliding','DS_DI_SlewPos','DS_DI_SlewNeg','Uppercarriage xd','Uppercarriage yd','Uppercarriage zd','MainBoom xd','MainBoom yd','MainBoom zd','DipperArm xd','DipperArm yd','DipperArm zd','Bucket xd','Bucket yd','Bucket zd','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','SoilTransferSensor2 SoilTot','soilTransferSensor SoilTot'],axis=1)
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','soilTransferSensor MassTot','Collisionsensor_Ground isColliding','DS_DI_SlewPos','DS_DI_SlewNeg','Uppercarriage xd','Uppercarriage yd','Uppercarriage zd','MainBoom xd','MainBoom yd','MainBoom zd','DipperArm xd','DipperArm yd','DipperArm zd','Bucket xd','Bucket yd','Bucket zd','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','SoilTransferSensor2 SoilTot','soilTransferSensor SoilTot'],axis=1)
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','soilTransferSensor MassTot','Collisionsensor_Ground isColliding','DS_DI_SlewPos','DS_DI_SlewNeg','Uppercarriage xd','Uppercarriage yd','Uppercarriage zd','MainBoom xd','MainBoom yd','MainBoom zd','DipperArm xd','DipperArm yd','DipperArm zd','Bucket xd','Bucket yd','Bucket zd','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','SoilTransferSensor2 SoilTot','soilTransferSensor SoilTot'],axis=1)
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','soilTransferSensor MassTot','Collisionsensor_Ground isColliding','DS_DI_SlewPos','DS_DI_SlewNeg','Uppercarriage xd','Uppercarriage yd','Uppercarriage zd','MainBoom xd','MainBoom yd','MainBoom zd','DipperArm xd','DipperArm yd','DipperArm zd','Bucket xd','Bucket yd','Bucket zd','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','SoilTransferSensor2 SoilTot','soilTransferSensor SoilTot'],axis=1)
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','soilTransferSensor MassTot','Collisionsensor_Ground isColliding','DS_DI_SlewPos','DS_DI_SlewNeg','Uppercarriage xd','Uppercarriage yd','Uppercarriage zd','MainBoom xd','MainBoom yd','MainBoom zd','DipperArm xd','DipperArm yd','DipperArm zd','Bucket xd','Bucket yd','Bucket zd','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','SoilTransferSensor2 SoilTot','soilTransferSensor SoilTot'],axis=1)
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','soilTransferSensor MassTot','Collisionsensor_Ground isColliding','DS_DI_SlewPos','DS_DI_SlewNeg','Uppercarriage xd','Uppercarriage yd','Uppercarriage zd','MainBoom xd','MainBoom yd','MainBoom zd','DipperArm xd','DipperArm yd','DipperArm zd','Bucket xd','Bucket yd','Bucket zd','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','SoilTransferSensor2 SoilTot','soilTransferSensor SoilTot'],axis=1)
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

X_pred = df1.drop(['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSDT','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','MassSensor7 Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','Time','Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue','MassHopper Mass'],axis=1)
y_actual = df1[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X_pred)
X_pred = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y_actual)
y_actual = pd.DataFrame(scaler_y)

X_train,X_test,y_train,y_test = train_test_split(X,y)
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2)
X_train3,X_test3,y_train3,y_test3 = train_test_split(X3,y3)
X_train4,X_test4,y_train4,y_test4 = train_test_split(X4,y4)
X_train5,X_test5,y_train5,y_test5 = train_test_split(X5,y5)
X_train6,X_test6,y_train6,y_test6 = train_test_split(X6,y6)
X_train7,X_test7,y_train7,y_test7 = train_test_split(X7,y7)
X_train8,X_test8,y_train8,y_test8 = train_test_split(X8,y8)
X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)

C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\an

In [92]:
#Variance based 
#Connected Weights 
# https://link.springer.com/chapter/10.1007/978-3-030-33778-0_24 
# https://github.com/rebelosa/feature-importance-neural-networks/blob/master/variance-based%20feature%20importance%20in%20artificial%20neural%20networks.ipynb
# VIANN
# Variance-based Feature Importance of Artificial Neural Networks
class VarImpVIANN(keras.callbacks.Callback):
    def __init__(self, verbose=0):
        self.verbose = verbose
        self.n = 0
        self.M2 = 0.0

    def on_train_begin(self, logs={}, verbose = 1):
        if self.verbose:
            print("VIANN version 1.0 (Wellford + Mean) update per epoch")
        self.diff = self.model.layers[0].get_weights()[0]
        
    def on_epoch_end(self, batch, logs={}):
        currentWeights = self.model.layers[0].get_weights()[0]
        
        self.n += 1
        delta = np.subtract(currentWeights, self.diff)
        self.diff += delta/self.n
        delta2 = np.subtract(currentWeights, self.diff)
        self.M2 += delta*delta2
            
        self.lastweights = self.model.layers[0].get_weights()[0]

    def on_train_end(self, batch, logs={}):
        if self.n < 2:
            self.s2 = float('nan')
        else:
            self.s2 = self.M2 / (self.n - 1)
        
        scores = np.sum(np.multiply(self.s2, np.abs(self.lastweights)), axis = 1)
        
        self.varScores = (scores - min(scores)) / (max(scores) - min(scores))
        if self.verbose:
            print("Most important variables: ",
                  np.array(self.varScores).argsort())

In [103]:
#Bulk all corr
resultlist = []
#drop = [0.2,0.3,0.4,0.5]

#Create the main Neural Network
model = Sequential()
#model.add(Dropout(j, input_shape=(41,)))
model.add(Dense(150,input_shape = (46,),activation='relu'))
model.add(Dense(25,activation='relu'))
model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

model.compile(loss='mse',optimizer='adam')

model.fit(X_train,y_train,batch_size=50,epochs=200,verbose=2,validation_data=(X_test,y_test),shuffle=False) #Changed shuffle to true
#model.fit(X_train2,y_train2,batch_size=50,epochs=200,verbose=2,validation_data=(X_test2,y_test2),shuffle=False) #Changed shuffle to true
model.fit(X_train3,y_train3,batch_size=50,epochs=200,verbose=2,validation_data=(X_test3,y_test3),shuffle=False) #Changed shuffle to true
#model.fit(X_train4,y_train4,batch_size=50,epochs=200,verbose=2,validation_data=(X_test4,y_test4),shuffle=False) #Changed shuffle to true
model.fit(X_train5,y_train5,batch_size=50,epochs=200,verbose=2,validation_data=(X_test5,y_test5),shuffle=False) #Changed shuffle to true
#model.fit(X_train6,y_train6,batch_size=50,epochs=200,verbose=2,validation_data=(X_test6,y_test6),shuffle=False) #Changed shuffle to true
model.fit(X_train7,y_train7,batch_size=50,epochs=200,verbose=2,validation_data=(X_test7,y_test7),shuffle=False) #Changed shuffle to true
#model.fit(X_train8,y_train8,batch_size=50,epochs=200,verbose=2,validation_data=(X_test8,y_test8),shuffle=False) #Changed shuffle to true
#model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
#model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
#model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
#model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
#model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
#model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
#model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 

y_pred = model.predict(X_train9)
bear = mse(y_train9,y_pred)

result = str(bear)+" ,layer 1 "+str(150) + " ,layer 2 "+str(25)
resultlist.append(pd.Series(result))


Train on 2139 samples, validate on 713 samples
Epoch 1/200
 - 1s - loss: 0.0475 - val_loss: 0.0250
Epoch 2/200
 - 0s - loss: 0.0193 - val_loss: 0.0141
Epoch 3/200
 - 0s - loss: 0.0128 - val_loss: 0.0107
Epoch 4/200
 - 0s - loss: 0.0102 - val_loss: 0.0089
Epoch 5/200
 - 0s - loss: 0.0087 - val_loss: 0.0078
Epoch 6/200
 - 0s - loss: 0.0078 - val_loss: 0.0071
Epoch 7/200
 - 0s - loss: 0.0071 - val_loss: 0.0066
Epoch 8/200
 - 0s - loss: 0.0066 - val_loss: 0.0062
Epoch 9/200
 - 0s - loss: 0.0062 - val_loss: 0.0058
Epoch 10/200
 - 0s - loss: 0.0058 - val_loss: 0.0056
Epoch 11/200
 - 0s - loss: 0.0056 - val_loss: 0.0053
Epoch 12/200
 - 0s - loss: 0.0053 - val_loss: 0.0050
Epoch 13/200
 - 0s - loss: 0.0051 - val_loss: 0.0048
Epoch 14/200
 - 0s - loss: 0.0049 - val_loss: 0.0046
Epoch 15/200
 - 0s - loss: 0.0047 - val_loss: 0.0044
Epoch 16/200
 - 0s - loss: 0.0045 - val_loss: 0.0042
Epoch 17/200
 - 0s - loss: 0.0044 - val_loss: 0.0041
Epoch 18/200
 - 0s - loss: 0.0042 - val_loss: 0.0039
Epoch 19

Epoch 154/200
 - 0s - loss: 0.0012 - val_loss: 0.0016
Epoch 155/200
 - 0s - loss: 0.0012 - val_loss: 0.0016
Epoch 156/200
 - 0s - loss: 0.0012 - val_loss: 0.0016
Epoch 157/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 158/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 159/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 160/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 161/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 162/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 163/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 164/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 165/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 166/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 167/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 168/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 169/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 170/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 171/200
 - 0s - loss: 0.0012 - val_loss: 0.0015
Epoch 172/200
 - 0s - loss: 

Epoch 106/200
 - 0s - loss: 9.3617e-04 - val_loss: 0.0012
Epoch 107/200
 - 0s - loss: 9.2989e-04 - val_loss: 0.0012
Epoch 108/200
 - 0s - loss: 9.2331e-04 - val_loss: 0.0012
Epoch 109/200
 - 0s - loss: 9.1775e-04 - val_loss: 0.0012
Epoch 110/200
 - 0s - loss: 9.1636e-04 - val_loss: 0.0011
Epoch 111/200
 - 0s - loss: 9.1281e-04 - val_loss: 0.0012
Epoch 112/200
 - 0s - loss: 9.0623e-04 - val_loss: 0.0011
Epoch 113/200
 - 0s - loss: 8.9770e-04 - val_loss: 0.0011
Epoch 114/200
 - 0s - loss: 8.9680e-04 - val_loss: 0.0011
Epoch 115/200
 - 0s - loss: 8.9152e-04 - val_loss: 0.0011
Epoch 116/200
 - 0s - loss: 8.9085e-04 - val_loss: 0.0011
Epoch 117/200
 - 0s - loss: 8.8505e-04 - val_loss: 0.0011
Epoch 118/200
 - 0s - loss: 8.7971e-04 - val_loss: 0.0011
Epoch 119/200
 - 0s - loss: 8.7803e-04 - val_loss: 0.0011
Epoch 120/200
 - 0s - loss: 8.7152e-04 - val_loss: 0.0011
Epoch 121/200
 - 0s - loss: 8.6937e-04 - val_loss: 0.0011
Epoch 122/200
 - 0s - loss: 8.6494e-04 - val_loss: 0.0011
Epoch 123/200


Epoch 47/200
 - 0s - loss: 9.4297e-04 - val_loss: 0.0011
Epoch 48/200
 - 0s - loss: 9.3300e-04 - val_loss: 0.0011
Epoch 49/200
 - 0s - loss: 9.2442e-04 - val_loss: 0.0011
Epoch 50/200
 - 0s - loss: 9.1080e-04 - val_loss: 0.0011
Epoch 51/200
 - 0s - loss: 9.1050e-04 - val_loss: 0.0011
Epoch 52/200
 - 0s - loss: 8.9209e-04 - val_loss: 0.0011
Epoch 53/200
 - 0s - loss: 8.8735e-04 - val_loss: 0.0011
Epoch 54/200
 - 0s - loss: 8.7613e-04 - val_loss: 0.0011
Epoch 55/200
 - 0s - loss: 8.6788e-04 - val_loss: 0.0011
Epoch 56/200
 - 0s - loss: 8.6076e-04 - val_loss: 0.0011
Epoch 57/200
 - 0s - loss: 8.4854e-04 - val_loss: 0.0011
Epoch 58/200
 - 0s - loss: 8.3820e-04 - val_loss: 0.0010
Epoch 59/200
 - 0s - loss: 8.2942e-04 - val_loss: 0.0010
Epoch 60/200
 - 0s - loss: 8.2031e-04 - val_loss: 0.0010
Epoch 61/200
 - 0s - loss: 8.1221e-04 - val_loss: 0.0010
Epoch 62/200
 - 0s - loss: 8.0570e-04 - val_loss: 0.0010
Epoch 63/200
 - 0s - loss: 7.9943e-04 - val_loss: 0.0010
Epoch 64/200
 - 0s - loss: 7.90

Epoch 182/200
 - 0s - loss: 4.4627e-04 - val_loss: 7.3177e-04
Epoch 183/200
 - 0s - loss: 4.4361e-04 - val_loss: 7.2085e-04
Epoch 184/200
 - 0s - loss: 4.4177e-04 - val_loss: 7.1979e-04
Epoch 185/200
 - 0s - loss: 4.3729e-04 - val_loss: 7.1548e-04
Epoch 186/200
 - 0s - loss: 4.3946e-04 - val_loss: 7.1629e-04
Epoch 187/200
 - 0s - loss: 4.4397e-04 - val_loss: 7.2258e-04
Epoch 188/200
 - 0s - loss: 4.3739e-04 - val_loss: 7.2479e-04
Epoch 189/200
 - 0s - loss: 4.4092e-04 - val_loss: 7.2212e-04
Epoch 190/200
 - 0s - loss: 4.3966e-04 - val_loss: 7.3246e-04
Epoch 191/200
 - 0s - loss: 4.3707e-04 - val_loss: 7.2414e-04
Epoch 192/200
 - 0s - loss: 4.2809e-04 - val_loss: 7.1475e-04
Epoch 193/200
 - 0s - loss: 4.3710e-04 - val_loss: 7.3034e-04
Epoch 194/200
 - 0s - loss: 4.2881e-04 - val_loss: 7.1914e-04
Epoch 195/200
 - 0s - loss: 4.3130e-04 - val_loss: 7.2040e-04
Epoch 196/200
 - 0s - loss: 4.3162e-04 - val_loss: 7.0845e-04
Epoch 197/200
 - 0s - loss: 4.3209e-04 - val_loss: 7.2996e-04
Epoch 19

Epoch 130/200
 - 0s - loss: 8.2119e-04 - val_loss: 0.0014
Epoch 131/200
 - 0s - loss: 8.0141e-04 - val_loss: 0.0014
Epoch 132/200
 - 0s - loss: 8.0665e-04 - val_loss: 0.0014
Epoch 133/200
 - 0s - loss: 7.9298e-04 - val_loss: 0.0014
Epoch 134/200
 - 0s - loss: 8.0830e-04 - val_loss: 0.0014
Epoch 135/200
 - 0s - loss: 8.2371e-04 - val_loss: 0.0013
Epoch 136/200
 - 0s - loss: 7.8977e-04 - val_loss: 0.0014
Epoch 137/200
 - 0s - loss: 7.8935e-04 - val_loss: 0.0014
Epoch 138/200
 - 0s - loss: 7.9776e-04 - val_loss: 0.0014
Epoch 139/200
 - 0s - loss: 8.0094e-04 - val_loss: 0.0014
Epoch 140/200
 - 0s - loss: 7.8312e-04 - val_loss: 0.0014
Epoch 141/200
 - 0s - loss: 7.8576e-04 - val_loss: 0.0014
Epoch 142/200
 - 0s - loss: 7.7294e-04 - val_loss: 0.0014
Epoch 143/200
 - 0s - loss: 7.7090e-04 - val_loss: 0.0014
Epoch 144/200
 - 0s - loss: 7.7075e-04 - val_loss: 0.0014
Epoch 145/200
 - 0s - loss: 7.6464e-04 - val_loss: 0.0014
Epoch 146/200
 - 0s - loss: 7.5787e-04 - val_loss: 0.0014
Epoch 147/200


ValueError: Error when checking input: expected dense_37_input to have shape (46,) but got array with shape (44,)

In [104]:
VIANN = VarImpVIANN(verbose=1)

model1 = model

model1.fit(X_train8, y_train8, validation_split=0.05, epochs=30, batch_size=64, shuffle=False, 
      verbose=1, callbacks = [VIANN])
      
print(VIANN.varScores)

Train on 1526 samples, validate on 81 samples
VIANN version 1.0 (Wellford + Mean) update per epoch
Epoch 1/30
1526/1526 [==============================] - 0s 33us/step - loss: 0.0545 - val_loss: 0.0179
Epoch 2/30
1526/1526 [==============================] - 0s 33us/step - loss: 0.0175 - val_loss: 0.0107
Epoch 3/30
1526/1526 [==============================] - 0s 31us/step - loss: 0.0125 - val_loss: 0.0090
Epoch 4/30
1526/1526 [==============================] - 0s 25us/step - loss: 0.0099 - val_loss: 0.0079
Epoch 5/30
1526/1526 [==============================] - 0s 26us/step - loss: 0.0090 - val_loss: 0.0073
Epoch 6/30
1526/1526 [==============================] - 0s 56us/step - loss: 0.0082 - val_loss: 0.0069
Epoch 7/30
1526/1526 [==============================] - 0s 25us/step - loss: 0.0076 - val_loss: 0.0065
Epoch 8/30
1526/1526 [==============================] - 0s 25us/step - loss: 0.0072 - val_loss: 0.0063
Epoch 9/30
1526/1526 [==============================] - 0s 25us/step - loss: 

In [77]:

lr = LinearRegression()
lr.fit(X9,y9[0])
lr.fit(X10,y10[0])
lr.fit(X11,y11[0])
lr.fit(X12,y12[0])
lr.fit(X13,y13[0])
lr.fit(X14,y14[0])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [78]:
rfe = RFE(estimator=lr,n_features_to_select=1,step=1,verbose=2)
rfe.fit(X15,y15[0])

#print("No.Features: %s" % (rfe.n_features))
print("Selected Features: %s" % (rfe.support_))
print("Feature Ranking: %s" % (rfe.ranking_))

Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 fe

In [79]:
#Permutation Importance
# https://www.kaggle.com/ourique/permutation-importance
# https://towardsdatascience.com/feature-importance-with-neural-network-346eb6205743
import sklearn.neural_network as sklnn

sklnn = MLPRegressor(hidden_layer_sizes=(50,100,50),activation='relu',solver='adam')

sklnn.fit(X_train9,y_train9)
sklnn.fit(X_train10,y_train10)
sklnn.fit(X_train11,y_train11)
sklnn.fit(X_train12,y_train12)
sklnn.fit(X_train13,y_train13)
sklnn.fit(X_train14,y_train14)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 100, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [82]:
perm = PermutationImportance(sklnn,random_state=1).fit(X_train15,y_train15)
eli5.show_weights(perm)


Weight,Feature
0.1359 ± 0.0071,x10
0.1053 ± 0.0049,x43
0.0757 ± 0.0035,x5
0.0667 ± 0.0027,x0
0.0628 ± 0.0015,x40
0.0588 ± 0.0022,x36
0.0490 ± 0.0038,x37
0.0394 ± 0.0017,x32
0.0216 ± 0.0028,x6
0.0214 ± 0.0014,x1


In [83]:
perm.feature_importances_

array([ 6.67340702e-02,  2.14064338e-02,  1.05724845e-02,  1.82411196e-02,
        1.99493503e-02,  7.57411972e-02,  2.15773554e-02,  5.83785225e-03,
       -1.26378128e-03,  1.92145952e-02,  1.35891384e-01,  1.19934417e-02,
        6.92653278e-03,  1.47010963e-02, -2.49148647e-03,  4.70309017e-03,
        4.33068648e-04,  1.06698111e-04,  3.02337765e-03,  6.21318026e-04,
        7.20440867e-05,  1.90102154e-03,  7.54681922e-04,  6.20650739e-03,
        2.27914294e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        4.57032274e-03,  1.30411766e-02,  3.40169429e-03,  3.64445740e-04,
        3.94461636e-02,  1.61155838e-03,  3.06790688e-03,  2.04220288e-02,
        5.88192448e-02,  4.89603614e-02,  1.40651586e-02,  1.52313724e-02,
        6.28499504e-02,  1.88792951e-02,  3.99374360e-03,  1.05336863e-01])

In [84]:
def LeaveOneFeatureOut(model, X, Y):
    OneOutScore = []
    n = X.shape[0]
    for i in range(0,X.shape[1]):
        newX = X.copy()
        newX[:,i] = 0 #np.random.normal(0,1,n)
        OneOutScore.append(model.evaluate(newX, Y, batch_size=2048, verbose=0))
    OneOutScore = pd.DataFrame(OneOutScore[:])
    ordered = np.argsort(-OneOutScore.iloc[:,0])
    return(OneOutScore, ordered)

In [85]:
L1FO = LeaveOneFeatureOut(model,X_train15.values,y_train15.values)

In [86]:
pd.set_option('display.max_rows', 500) 
print(L1FO)

(           0
0   0.019887
1   0.007510
2   0.004342
3   0.004680
4   0.015352
5   0.007255
6   0.008374
7   0.007805
8   0.005894
9   0.005138
10  0.006949
11  0.001653
12  0.001765
13  0.003725
14  0.003919
15  0.012696
16  0.001437
17  0.001646
18  0.005070
19  0.010458
20  0.002143
21  0.002312
22  0.002239
23  0.002386
24  0.002979
25  0.000555
26  0.000555
27  0.000555
28  0.003481
29  0.003806
30  0.002545
31  0.001700
32  0.003724
33  0.026397
34  0.002803
35  0.004173
36  0.021382
37  0.003020
38  0.004792
39  0.004266
40  0.008713
41  0.005953
42  0.007362
43  0.012252, 0     33
1     36
2      0
3      4
4     15
5     43
6     19
7     40
8      6
9      7
10     1
11    42
12     5
13    10
14    41
15     8
16     9
17    18
18    38
19     3
20     2
21    39
22    35
23    14
24    29
25    13
26    32
27    28
28    37
29    24
30    34
31    30
32    23
33    21
34    22
35    20
36    12
37    31
38    11
39    17
40    16
41    26
42    25
43    27
Name: 0, dtype: i

In [ ]:
#VIANN 
X_10 = df[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y_10 = df[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X10=MinMaxScaler()
scaler_X10 = scaler_X10.fit_transform(X_10)
X_10 = pd.DataFrame(scaler_X10)
scaler_y10=MinMaxScaler()
scaler_y10 = scaler_y10.fit_transform(y_10)
y_10 = pd.DataFrame(scaler_y10)

X_pred10 = df1[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y_actual10 = df1[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X_pred10)
X_pred10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y_actual10)
y_actual10 = pd.DataFrame(scaler_y)


X_train,X_test,y_train,y_test = train_test_split(X_10,y_10)


In [ ]:
#PI Neural Network decorrelated with 10 features

neurons = [5,10,25,50,75,100,150,200]
decorrdata10 = []
#drop = [0.2,0.3,0.4,0.5]
for x in neurons:
    for i in neurons:

        #Create the main Neural Network
        model = Sequential()
        model.add(Dense(x, activation = 'relu',input_shape=(19,)))
        model.add(Dense(i,activation='relu'))
        model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

        model.compile(loss='mse',optimizer='adam')

        model.fit(X_train,y_train,batch_size=50,epochs=200,verbose=2,validation_data=(X_test,y_test),shuffle=False) #Changed shuffle to true

        y_pred = model.predict(X_pred)
        bear = mse(y_actual,y_pred)

        result = str(bear)+" ,layer 1 "+str(x) + " ,layer 2 "+str(i)
        decorrdata10.append(pd.Series(result))

In [ ]:

df9= pd.read_csv('2ex9.csv',header=0)
df10= pd.read_csv('2ex10.csv',header=0)
df11= pd.read_csv('2ex11.csv',header=0)
df12= pd.read_csv('2ex12.csv',header=0)
df13= pd.read_csv('2ex13.csv',header=0)
df14= pd.read_csv('2ex14.csv',header=0)
df15= pd.read_csv('2ex15.csv',header=0)



X9 = df9[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_Fuel_Meter','DS_BucketCylinderPressureB','DS_CylinderBucketLift','DS_Slew']]
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

#X_pred = df1.drop(df1.ix[:,69:],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,52:57],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,25:40],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,17])
#y_actual = df1.iloc[:,25:29]

X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)
#L1FO Neural Network decorrelated with 10 features


In [ ]:
#10 Inputs
neurons = [5,10,25,50,75,100,150,200]
imuresultlist = []
#drop = [0.2,0.3,0.4,0.5]
for x in neurons:
    for i in neurons:
        #for j in drop:

            #Create the main Neural Network
            model = Sequential()
            #model.add(Dropout(j, input_shape=(41,)))
            model.add(Dense(x,input_shape = (20,),activation='relu'))
            model.add(Dense(i,activation='relu'))
            model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

            model.compile(loss='mse',optimizer='adam')

            #model.fit(X_train,y_train,batch_size=50,epochs=200,verbose=2,validation_data=(X_test,y_test),shuffle=False) #Changed shuffle to true
            #model.fit(X_train2,y_train2,batch_size=50,epochs=200,verbose=2,validation_data=(X_test2,y_test2),shuffle=False) #Changed shuffle to true
            #model.fit(X_train3,y_train3,batch_size=50,epochs=200,verbose=2,validation_data=(X_test3,y_test3),shuffle=False) #Changed shuffle to true
            #model.fit(X_train4,y_train4,batch_size=50,epochs=200,verbose=2,validation_data=(X_test4,y_test4),shuffle=False) #Changed shuffle to true
            #model.fit(X_train5,y_train5,batch_size=50,epochs=200,verbose=2,validation_data=(X_test5,y_test5),shuffle=False) #Changed shuffle to true
            #model.fit(X_train6,y_train6,batch_size=50,epochs=200,verbose=2,validation_data=(X_test6,y_test6),shuffle=False) #Changed shuffle to true
            #model.fit(X_train7,y_train7,batch_size=50,epochs=200,verbose=2,validation_data=(X_test7,y_test7),shuffle=False) #Changed shuffle to true
            #model.fit(X_train8,y_train8,batch_size=50,epochs=200,verbose=2,validation_data=(X_test8,y_test8),shuffle=False) #Changed shuffle to true
            model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
            model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
            model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
            model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
            model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
            model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
            #model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 

            y_pred = model.predict(X_train15)
            bear = mse(y_train15,y_pred)

            result = str(bear)+" ,layer 1 "+str(x) + " ,layer 2 "+str(i)
            imuresultlist.append(pd.Series(result))


In [ ]:
np.set_printoptions(threshold=np.inf)
print(imuresultlist)

In [112]:
#L1FO 

df9= pd.read_csv('2ex9.csv',header=0)
df10= pd.read_csv('2ex10.csv',header=0)
df11= pd.read_csv('2ex11.csv',header=0)
df12= pd.read_csv('2ex12.csv',header=0)
df13= pd.read_csv('2ex13.csv',header=0)
df14= pd.read_csv('2ex14.csv',header=0)
df15= pd.read_csv('2ex15.csv',header=0)



X9 = df9[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_CylinderBucketLift']]
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_CylinderBucketLift']]
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_CylinderBucketLift']]
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_CylinderBucketLift']]
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_CylinderBucketLift']]
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_CylinderBucketLift']]
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15[['DS_DST1','DS_DST2','DS_DST3','DS_DST4','DS_DST5','DS_DST6','DS_DST7','DS_DST8','DS_DSD1','DS_DSD2','DS_DSD3','DS_DSD4','massSensorBucketTeeth Mass','SoilTransferSensor2 MassTot','Collisionsensor_Ground nminor','Collisionsensor_Ground nserious','Collisionsensor_Ground ncritical','DS_MotorRPM','DS_Fuel_Meter','DS_CylinderBucketLift']]
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

#X_pred = df1.drop(df1.ix[:,69:],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,52:57],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,25:40],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,17])
#y_actual = df1.iloc[:,25:29]

X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)
#PI Neural Network decorrelated with 10 features


#PINeural Network decorrelated with 30 features

result20=[]
#Create the main Neural Network
for i in range(5):
    model = Sequential()
    model.add(Dense(50, activation = 'relu',input_shape=(20,)))
    model.add(Dense(5,activation='relu'))
    model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

    model.compile(loss='mse',optimizer='adam')

    model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
    model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
    model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
    model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
    model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
    model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
    #model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 

    y_pred = model.predict(X_train15)
    result30.append([mse(y_train15,y_pred),mae(y_train15,y_pred),r2(y_train15,y_pred)])


C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\an

Train on 2966 samples, validate on 989 samples
Epoch 1/200
 - 4s - loss: 0.0457 - val_loss: 0.0357
Epoch 2/200
 - 0s - loss: 0.0298 - val_loss: 0.0260
Epoch 3/200
 - 0s - loss: 0.0229 - val_loss: 0.0218
Epoch 4/200
 - 0s - loss: 0.0194 - val_loss: 0.0191
Epoch 5/200
 - 0s - loss: 0.0168 - val_loss: 0.0169
Epoch 6/200
 - 0s - loss: 0.0148 - val_loss: 0.0151
Epoch 7/200
 - 0s - loss: 0.0132 - val_loss: 0.0136
Epoch 8/200
 - 0s - loss: 0.0119 - val_loss: 0.0125
Epoch 9/200
 - 0s - loss: 0.0111 - val_loss: 0.0117
Epoch 10/200
 - 0s - loss: 0.0104 - val_loss: 0.0110
Epoch 11/200
 - 0s - loss: 0.0099 - val_loss: 0.0105
Epoch 12/200
 - 0s - loss: 0.0095 - val_loss: 0.0101
Epoch 13/200
 - 0s - loss: 0.0091 - val_loss: 0.0097
Epoch 14/200
 - 0s - loss: 0.0088 - val_loss: 0.0094
Epoch 15/200
 - 0s - loss: 0.0085 - val_loss: 0.0091
Epoch 16/200
 - 0s - loss: 0.0083 - val_loss: 0.0088
Epoch 17/200
 - 0s - loss: 0.0081 - val_loss: 0.0086
Epoch 18/200
 - 0s - loss: 0.0079 - val_loss: 0.0084
Epoch 19

Epoch 154/200
 - 0s - loss: 0.0032 - val_loss: 0.0035
Epoch 155/200
 - 0s - loss: 0.0032 - val_loss: 0.0035
Epoch 156/200
 - 0s - loss: 0.0032 - val_loss: 0.0035
Epoch 157/200
 - 0s - loss: 0.0032 - val_loss: 0.0035
Epoch 158/200
 - 0s - loss: 0.0032 - val_loss: 0.0035
Epoch 159/200
 - 0s - loss: 0.0032 - val_loss: 0.0035
Epoch 160/200
 - 0s - loss: 0.0032 - val_loss: 0.0035
Epoch 161/200
 - 0s - loss: 0.0031 - val_loss: 0.0035
Epoch 162/200
 - 0s - loss: 0.0031 - val_loss: 0.0035
Epoch 163/200
 - 0s - loss: 0.0031 - val_loss: 0.0035
Epoch 164/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 165/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 166/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 167/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 168/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 169/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 170/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 171/200
 - 0s - loss: 0.0031 - val_loss: 0.0034
Epoch 172/200
 - 0s - loss: 

Epoch 107/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 108/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 109/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 110/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 111/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 112/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 113/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 114/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 115/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 116/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 117/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 118/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 119/200
 - 0s - loss: 0.0045 - val_loss: 0.0046
Epoch 120/200
 - 0s - loss: 0.0044 - val_loss: 0.0046
Epoch 121/200
 - 0s - loss: 0.0044 - val_loss: 0.0046
Epoch 122/200
 - 0s - loss: 0.0044 - val_loss: 0.0045
Epoch 123/200
 - 0s - loss: 0.0044 - val_loss: 0.0045
Epoch 124/200
 - 0s - loss: 0.0044 - val_loss: 0.0045
Epoch 125/200
 - 0s - loss: 

 - 0s - loss: 0.0030 - val_loss: 0.0034
Epoch 60/200
 - 0s - loss: 0.0029 - val_loss: 0.0034
Epoch 61/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 62/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 63/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 64/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 65/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 66/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 67/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 68/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 69/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 70/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 71/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 72/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 73/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 74/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 75/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 76/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 77/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 

 - 0s - loss: 0.0038 - val_loss: 0.0042
Epoch 12/200
 - 0s - loss: 0.0038 - val_loss: 0.0042
Epoch 13/200
 - 0s - loss: 0.0037 - val_loss: 0.0041
Epoch 14/200
 - 0s - loss: 0.0037 - val_loss: 0.0041
Epoch 15/200
 - 0s - loss: 0.0037 - val_loss: 0.0041
Epoch 16/200
 - 0s - loss: 0.0036 - val_loss: 0.0040
Epoch 17/200
 - 0s - loss: 0.0036 - val_loss: 0.0040
Epoch 18/200
 - 0s - loss: 0.0036 - val_loss: 0.0040
Epoch 19/200
 - 0s - loss: 0.0035 - val_loss: 0.0040
Epoch 20/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 21/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 22/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 23/200
 - 0s - loss: 0.0034 - val_loss: 0.0039
Epoch 24/200
 - 0s - loss: 0.0034 - val_loss: 0.0038
Epoch 25/200
 - 0s - loss: 0.0034 - val_loss: 0.0038
Epoch 26/200
 - 0s - loss: 0.0034 - val_loss: 0.0038
Epoch 27/200
 - 0s - loss: 0.0034 - val_loss: 0.0038
Epoch 28/200
 - 0s - loss: 0.0033 - val_loss: 0.0038
Epoch 29/200
 - 0s - loss: 0.0033 - val_loss: 0.0037
Epoch 

Epoch 165/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 166/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 167/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 168/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 169/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 170/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 171/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 172/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 173/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 174/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 175/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 176/200
 - 0s - loss: 0.0023 - val_loss: 0.0025
Epoch 177/200
 - 0s - loss: 0.0022 - val_loss: 0.0025
Epoch 178/200
 - 0s - loss: 0.0022 - val_loss: 0.0025
Epoch 179/200
 - 0s - loss: 0.0022 - val_loss: 0.0025
Epoch 180/200
 - 0s - loss: 0.0022 - val_loss: 0.0025
Epoch 181/200
 - 0s - loss: 0.0022 - val_loss: 0.0025
Epoch 182/200
 - 0s - loss: 0.0022 - val_loss: 0.0025
Epoch 183/200
 - 0s - loss: 

Epoch 118/200
 - 0s - loss: 0.0049 - val_loss: 0.0045
Epoch 119/200
 - 0s - loss: 0.0049 - val_loss: 0.0045
Epoch 120/200
 - 0s - loss: 0.0049 - val_loss: 0.0045
Epoch 121/200
 - 0s - loss: 0.0049 - val_loss: 0.0045
Epoch 122/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 123/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 124/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 125/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 126/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 127/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 128/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 129/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 130/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 131/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 132/200
 - 0s - loss: 0.0048 - val_loss: 0.0045
Epoch 133/200
 - 0s - loss: 0.0047 - val_loss: 0.0045
Epoch 134/200
 - 0s - loss: 0.0047 - val_loss: 0.0045
Epoch 135/200
 - 0s - loss: 0.0047 - val_loss: 0.0044
Epoch 136/200
 - 0s - loss: 

KeyboardInterrupt: 

In [110]:
print(result30)

[[0.0167012969606313, 0.09348999179516598, 0.4312771752331501], [0.020978056427056017, 0.09932893464778521, 0.31222476048990655], [0.01809864733653867, 0.09598213049518085, 0.3644141101301289], [0.019471600428982286, 0.09545637992507658, 0.4201118140816633], [0.01950818454857268, 0.10075209661219732, 0.3906503879554519]]


In [34]:
#L1FO 

X9 = df9[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle','Collisionsensor_Ground nminor','DS_Boom_Angle','DS_Slew','DS_BucketCylinderPressureB','DS_DSD1','DS_DSD2','SoilTransferSensor2 MassTot','DS_CylinderBoomLiftL','DS_Upperwz','massSensorBucketTeeth Mass']]
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle','Collisionsensor_Ground nminor','DS_Boom_Angle','DS_Slew','DS_BucketCylinderPressureB','DS_DSD1','DS_DSD2','SoilTransferSensor2 MassTot','DS_CylinderBoomLiftL','DS_Upperwz','massSensorBucketTeeth Mass']]
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle','Collisionsensor_Ground nminor','DS_Boom_Angle','DS_Slew','DS_BucketCylinderPressureB','DS_DSD1','DS_DSD2','SoilTransferSensor2 MassTot','DS_CylinderBoomLiftL','DS_Upperwz','massSensorBucketTeeth Mass']]
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle','Collisionsensor_Ground nminor','DS_Boom_Angle','DS_Slew','DS_BucketCylinderPressureB','DS_DSD1','DS_DSD2','SoilTransferSensor2 MassTot','DS_CylinderBoomLiftL','DS_Upperwz','massSensorBucketTeeth Mass']]
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle','Collisionsensor_Ground nminor','DS_Boom_Angle','DS_Slew','DS_BucketCylinderPressureB','DS_DSD1','DS_DSD2','SoilTransferSensor2 MassTot','DS_CylinderBoomLiftL','DS_Upperwz','massSensorBucketTeeth Mass']]
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle','Collisionsensor_Ground nminor','DS_Boom_Angle','DS_Slew','DS_BucketCylinderPressureB','DS_DSD1','DS_DSD2','SoilTransferSensor2 MassTot','DS_CylinderBoomLiftL','DS_Upperwz','massSensorBucketTeeth Mass']]
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle','Collisionsensor_Ground nminor','DS_Boom_Angle','DS_Slew','DS_BucketCylinderPressureB','DS_DSD1','DS_DSD2','SoilTransferSensor2 MassTot','DS_CylinderBoomLiftL','DS_Upperwz','massSensorBucketTeeth Mass']]
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

#X_pred = df1.drop(df1.ix[:,69:],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,52:57],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,25:40],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,17])
#y_actual = df1.iloc[:,25:29]

#X_train,X_test,y_train,y_test = train_test_split(X_20,y_20)
X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)

#L1FO Neural Network decorrelated with 30 features

result20=[]
#Create the main Neural Network
for i in range(5):
    model = Sequential()
    model.add(Dense(50, activation = 'relu',input_shape=(30,)))
    model.add(Dense(5,activation='relu'))
    model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

    model.compile(loss='mse',optimizer='adam')

    model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
    model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
    model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
    model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
    #model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
    #model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
    #model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 

    y_pred = model.predict(X_train15)
    result20.append([mse(y_train15,y_pred),mae(y_train15,y_pred),r2(y_train15,y_pred)])

C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\an

Train on 2966 samples, validate on 989 samples
Epoch 1/200
 - 5s - loss: 0.0367 - val_loss: 0.0291
Epoch 2/200
 - 0s - loss: 0.0238 - val_loss: 0.0201
Epoch 3/200
 - 0s - loss: 0.0163 - val_loss: 0.0141
Epoch 4/200
 - 0s - loss: 0.0120 - val_loss: 0.0114
Epoch 5/200
 - 0s - loss: 0.0101 - val_loss: 0.0101
Epoch 6/200
 - 0s - loss: 0.0090 - val_loss: 0.0093
Epoch 7/200
 - 0s - loss: 0.0082 - val_loss: 0.0087
Epoch 8/200
 - 0s - loss: 0.0077 - val_loss: 0.0083
Epoch 9/200
 - 0s - loss: 0.0074 - val_loss: 0.0079
Epoch 10/200
 - 0s - loss: 0.0070 - val_loss: 0.0076
Epoch 11/200
 - 0s - loss: 0.0068 - val_loss: 0.0074
Epoch 12/200
 - 0s - loss: 0.0066 - val_loss: 0.0072
Epoch 13/200
 - 0s - loss: 0.0064 - val_loss: 0.0070
Epoch 14/200
 - 0s - loss: 0.0062 - val_loss: 0.0068
Epoch 15/200
 - 0s - loss: 0.0060 - val_loss: 0.0067
Epoch 16/200
 - 0s - loss: 0.0059 - val_loss: 0.0065
Epoch 17/200
 - 0s - loss: 0.0058 - val_loss: 0.0064
Epoch 18/200
 - 0s - loss: 0.0056 - val_loss: 0.0063
Epoch 19

Epoch 154/200
 - 0s - loss: 0.0027 - val_loss: 0.0031
Epoch 155/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 156/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 157/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 158/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 159/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 160/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 161/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 162/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 163/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 164/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 165/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 166/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 167/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 168/200
 - 0s - loss: 0.0026 - val_loss: 0.0031
Epoch 169/200
 - 0s - loss: 0.0026 - val_loss: 0.0030
Epoch 170/200
 - 0s - loss: 0.0026 - val_loss: 0.0030
Epoch 171/200
 - 0s - loss: 0.0026 - val_loss: 0.0030
Epoch 172/200
 - 0s - loss: 

Epoch 107/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 108/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 109/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 110/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 111/200
 - 0s - loss: 0.0026 - val_loss: 0.0030
Epoch 112/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 113/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 114/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 115/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 116/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 117/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 118/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 119/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 120/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 121/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 122/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 123/200
 - 0s - loss: 0.0026 - val_loss: 0.0029
Epoch 124/200
 - 0s - loss: 0.0025 - val_loss: 0.0028
Epoch 125/200
 - 0s - loss: 

 - 0s - loss: 0.0022 - val_loss: 0.0024
Epoch 60/200
 - 0s - loss: 0.0022 - val_loss: 0.0024
Epoch 61/200
 - 0s - loss: 0.0022 - val_loss: 0.0024
Epoch 62/200
 - 0s - loss: 0.0022 - val_loss: 0.0024
Epoch 63/200
 - 0s - loss: 0.0022 - val_loss: 0.0024
Epoch 64/200
 - 0s - loss: 0.0021 - val_loss: 0.0024
Epoch 65/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 66/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 67/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 68/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 69/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 70/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 71/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 72/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 73/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 74/200
 - 0s - loss: 0.0021 - val_loss: 0.0023
Epoch 75/200
 - 0s - loss: 0.0020 - val_loss: 0.0022
Epoch 76/200
 - 0s - loss: 0.0020 - val_loss: 0.0022
Epoch 77/200
 - 0s - loss: 0.0020 - val_loss: 0.0022
Epoch 

...

Epoch 155/200
 - 0s - loss: 0.0044 - val_loss: 0.0051
Epoch 156/200
 - 0s - loss: 0.0044 - val_loss: 0.0051
Epoch 157/200
 - 0s - loss: 0.0044 - val_loss: 0.0051
Epoch 158/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 159/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 160/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 161/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 162/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 163/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 164/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 165/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 166/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 167/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 168/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 169/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 170/200
 - 0s - loss: 0.0043 - val_loss: 0.0050
Epoch 171/200
 - 0s - loss: 0.0043 - val_loss: 0.0049
Epoch 172/200
 - 0s - loss: 0.0043 - val_loss: 0.0049
Epoch 173/200
 - 0s - loss: 

Epoch 108/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 109/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 110/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 111/200
 - 1s - loss: 0.0059 - val_loss: 0.0061
Epoch 112/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 113/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 114/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 115/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 116/200
 - 0s - loss: 0.0059 - val_loss: 0.0061
Epoch 117/200
 - 0s - loss: 0.0059 - val_loss: 0.0060
Epoch 118/200
 - 0s - loss: 0.0058 - val_loss: 0.0060
Epoch 119/200
 - 0s - loss: 0.0058 - val_loss: 0.0060
Epoch 120/200
 - 0s - loss: 0.0058 - val_loss: 0.0060
Epoch 121/200
 - 1s - loss: 0.0058 - val_loss: 0.0060
Epoch 122/200
 - 0s - loss: 0.0058 - val_loss: 0.0060
Epoch 123/200
 - 0s - loss: 0.0058 - val_loss: 0.0060
Epoch 124/200
 - 0s - loss: 0.0058 - val_loss: 0.0060
Epoch 125/200
 - 0s - loss: 0.0058 - val_loss: 0.0060
Epoch 126/200
 - 1s - loss: 

 - 0s - loss: 0.0036 - val_loss: 0.0039
Epoch 61/200
 - 0s - loss: 0.0036 - val_loss: 0.0039
Epoch 62/200
 - 0s - loss: 0.0036 - val_loss: 0.0039
Epoch 63/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 64/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 65/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 66/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 67/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 68/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 69/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 70/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 71/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 72/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 73/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 74/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 75/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 76/200
 - 0s - loss: 0.0035 - val_loss: 0.0039
Epoch 77/200
 - 0s - loss: 0.0035 - val_loss: 0.0038
Epoch 78/200
 - 0s - loss: 0.0035 - val_loss: 0.0038
Epoch 

 - 0s - loss: 0.0031 - val_loss: 0.0037
Epoch 13/200
 - 0s - loss: 0.0031 - val_loss: 0.0036
Epoch 14/200
 - 0s - loss: 0.0031 - val_loss: 0.0036
Epoch 15/200
 - 0s - loss: 0.0031 - val_loss: 0.0036
Epoch 16/200
 - 0s - loss: 0.0031 - val_loss: 0.0035
Epoch 17/200
 - 0s - loss: 0.0030 - val_loss: 0.0035
Epoch 18/200
 - 0s - loss: 0.0030 - val_loss: 0.0035
Epoch 19/200
 - 0s - loss: 0.0030 - val_loss: 0.0035
Epoch 20/200
 - 0s - loss: 0.0030 - val_loss: 0.0035
Epoch 21/200
 - 0s - loss: 0.0030 - val_loss: 0.0034
Epoch 22/200
 - 0s - loss: 0.0029 - val_loss: 0.0034
Epoch 23/200
 - 0s - loss: 0.0029 - val_loss: 0.0034
Epoch 24/200
 - 0s - loss: 0.0029 - val_loss: 0.0034
Epoch 25/200
 - 0s - loss: 0.0029 - val_loss: 0.0034
Epoch 26/200
 - 0s - loss: 0.0029 - val_loss: 0.0034
Epoch 27/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 28/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 29/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 30/200
 - 0s - loss: 0.0028 - val_loss: 0.0033
Epoch 

Epoch 166/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 167/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 168/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 169/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 170/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 171/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 172/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 173/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 174/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 175/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 176/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 177/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 178/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 179/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 180/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 181/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 182/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 183/200
 - 0s - loss: 0.0023 - val_loss: 0.0028
Epoch 184/200
 - 0s - loss: 

In [35]:
print(result20)

[[0.024272884398246923, 0.10910490026917184, 0.3083354319026014], [0.027973023643001794, 0.12044803898130947, 0.2672603051498438], [0.032614031103893944, 0.13188447668013092, 0.1757029620309211], [0.024881616844819158, 0.11418339779018127, 0.30233404237648837], [0.025329309648042298, 0.11977852277724015, 0.33593897930748]]


In [37]:
#L1FO 

X9 = df9[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle']]
y9 = df9[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X9)
X9 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y9)
y9 = pd.DataFrame(scaler_y)

X10 = df10[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle']]
y10 = df10[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X10)
X10 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y10)
y10 = pd.DataFrame(scaler_y)

X11 = df11[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle']]
y11 = df11[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X11)
X11 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y11)
y11 = pd.DataFrame(scaler_y)

X12 = df12[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle']]
y12 = df12[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X12)
X12 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y12)
y12 = pd.DataFrame(scaler_y)

X13 = df13[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle']]
y13 = df13[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X13)
X13 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y13)
y13 = pd.DataFrame(scaler_y)

X14 = df14[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle']]
y14 = df14[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X14)
X14 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y14)
y14 = pd.DataFrame(scaler_y)

X15 = df15[['DS_Boomwz','DS_MotorRPM','DS_Dipperwz','DS_DipperCylinderPressureB','DS_Slew_Force','Collisionsensor_Ground nserious','DS_CylinderBucketLift','DS_Dipper_xdd','DS_BoomCylinderPressureB','DS_Bucket_Angle']]
y15 = df15[['Input_Slew RealValue','Input_BoomLift RealValue','Input_DipperArm RealValue','Input_Bucket RealValue']]
scaler_X=MinMaxScaler()
scaler_X = scaler_X.fit_transform(X15)
X15 = pd.DataFrame(scaler_X)
scaler_y=MinMaxScaler()
scaler_y = scaler_y.fit_transform(y15)
y15 = pd.DataFrame(scaler_y)

#X_pred = df1.drop(df1.ix[:,69:],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,52:57],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,25:40],axis=1)
#X_pred = X_pred.drop(X_pred.ix[:,17])
#y_actual = df1.iloc[:,25:29]

#X_train,X_test,y_train,y_test = train_test_split(X_20,y_20)
X_train9,X_test9,y_train9,y_test9 = train_test_split(X9,y9)
X_train10,X_test10,y_train10,y_test10 = train_test_split(X10,y10)
X_train11,X_test11,y_train11,y_test11 = train_test_split(X11,y11)
X_train12,X_test12,y_train12,y_test12 = train_test_split(X12,y12)
X_train13,X_test13,y_train13,y_test13 = train_test_split(X13,y13)
X_train14,X_test14,y_train14,y_test14 = train_test_split(X14,y14)
X_train15,X_test15,y_train15,y_test15 = train_test_split(X15,y15)

#L1FO Neural Network decorrelated with 30 features

result10=[]
#Create the main Neural Network
for i in range(5):
    model = Sequential()
    model.add(Dense(200, activation = 'relu',input_shape=(10,)))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(4,activation='sigmoid'))#Changed to sigmoid to allow 0 to 1

    model.compile(loss='mse',optimizer='adam')

    model.fit(X_train9,y_train9,batch_size=50,epochs=200,verbose=2,validation_data=(X_test9,y_test9),shuffle=False) #Changed shuffle to true
    model.fit(X_train10,y_train10,batch_size=50,epochs=200,verbose=2,validation_data=(X_test10,y_test10),shuffle=False) #Changed shuffle to true
    model.fit(X_train11,y_train11,batch_size=50,epochs=200,verbose=2,validation_data=(X_test11,y_test11),shuffle=False) #Changed shuffle to true
    model.fit(X_train12,y_train12,batch_size=50,epochs=200,verbose=2,validation_data=(X_test12,y_test12),shuffle=False) #Changed shuffle to true
    #model.fit(X_train13,y_train13,batch_size=50,epochs=200,verbose=2,validation_data=(X_test13,y_test13),shuffle=False) #Changed shuffle to true
    #model.fit(X_train14,y_train14,batch_size=50,epochs=200,verbose=2,validation_data=(X_test14,y_test14),shuffle=False) #Changed shuffle to true
    #model.fit(X_train15,y_train15,batch_size=50,epochs=200,verbose=2,validation_data=(X_test15,y_test15),shuffle=False) #Changed shuffle to true 

    y_pred = model.predict(X_train15)
    result10.append([mse(y_train15,y_pred),mae(y_train15,y_pred),r2(y_train15,y_pred)])


C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\wsjr\AppData\Local\Continuum\an

Train on 2966 samples, validate on 989 samples
Epoch 1/200
 - 5s - loss: 0.0338 - val_loss: 0.0196
Epoch 2/200
 - 0s - loss: 0.0174 - val_loss: 0.0142
Epoch 3/200
 - 0s - loss: 0.0138 - val_loss: 0.0121
Epoch 4/200
 - 0s - loss: 0.0120 - val_loss: 0.0107
Epoch 5/200
 - 0s - loss: 0.0107 - val_loss: 0.0098
Epoch 6/200
 - 0s - loss: 0.0098 - val_loss: 0.0091
Epoch 7/200
 - 0s - loss: 0.0091 - val_loss: 0.0087
Epoch 8/200
 - 0s - loss: 0.0086 - val_loss: 0.0082
Epoch 9/200
 - 0s - loss: 0.0082 - val_loss: 0.0079
Epoch 10/200
 - 0s - loss: 0.0079 - val_loss: 0.0075
Epoch 11/200
 - 0s - loss: 0.0076 - val_loss: 0.0072
Epoch 12/200
 - 0s - loss: 0.0073 - val_loss: 0.0069
Epoch 13/200
 - 0s - loss: 0.0071 - val_loss: 0.0067
Epoch 14/200
 - 0s - loss: 0.0068 - val_loss: 0.0064
Epoch 15/200
 - 0s - loss: 0.0066 - val_loss: 0.0062
Epoch 16/200
 - 0s - loss: 0.0065 - val_loss: 0.0060
Epoch 17/200
 - 0s - loss: 0.0063 - val_loss: 0.0059
Epoch 18/200
 - 0s - loss: 0.0061 - val_loss: 0.0057
Epoch 19

Epoch 154/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 155/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 156/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 157/200
 - 0s - loss: 0.0028 - val_loss: 0.0030
Epoch 158/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 159/200
 - 0s - loss: 0.0028 - val_loss: 0.0030
Epoch 160/200
 - 0s - loss: 0.0028 - val_loss: 0.0030
Epoch 161/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 162/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 163/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 164/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 165/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 166/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 167/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 168/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 169/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 170/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 171/200
 - 0s - loss: 0.0027 - val_loss: 0.0030
Epoch 172/200
 - 0s - loss: 

Epoch 107/200
 - 0s - loss: 0.0036 - val_loss: 0.0042
Epoch 108/200
 - 0s - loss: 0.0036 - val_loss: 0.0042
Epoch 109/200
 - 0s - loss: 0.0036 - val_loss: 0.0042
Epoch 110/200
 - 0s - loss: 0.0036 - val_loss: 0.0041
Epoch 111/200
 - 0s - loss: 0.0035 - val_loss: 0.0042
Epoch 112/200
 - 0s - loss: 0.0035 - val_loss: 0.0042
Epoch 113/200
 - 0s - loss: 0.0035 - val_loss: 0.0042
Epoch 114/200
 - 0s - loss: 0.0035 - val_loss: 0.0042
Epoch 115/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 116/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 117/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 118/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 119/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 120/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 121/200
 - 1s - loss: 0.0035 - val_loss: 0.0041
Epoch 122/200
 - 1s - loss: 0.0035 - val_loss: 0.0041
Epoch 123/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 124/200
 - 0s - loss: 0.0035 - val_loss: 0.0041
Epoch 125/200
 - 0s - loss: 

 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 60/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 61/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 62/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 63/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 64/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 65/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 66/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 67/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 68/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 69/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 70/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 71/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 72/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 73/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 74/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 75/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 76/200
 - 0s - loss: 0.0027 - val_loss: 0.0031
Epoch 77/200
 - 0s - loss: 0.0027 - val_loss: 0.0031
Epoch 

...

Epoch 162/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 163/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 164/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 165/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 166/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 167/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 168/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 169/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 170/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 171/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 172/200
 - 0s - loss: 0.0029 - val_loss: 0.0031
Epoch 173/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 174/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 175/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 176/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 177/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 178/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 179/200
 - 0s - loss: 0.0028 - val_loss: 0.0031
Epoch 180/200
 - 0s - loss: 

Epoch 115/200
 - 0s - loss: 0.0042 - val_loss: 0.0046
Epoch 116/200
 - 0s - loss: 0.0042 - val_loss: 0.0046
Epoch 117/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 118/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 119/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 120/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 121/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 122/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 123/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 124/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 125/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 126/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 127/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 128/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 129/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 130/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 131/200
 - 0s - loss: 0.0041 - val_loss: 0.0046
Epoch 132/200
 - 0s - loss: 0.0040 - val_loss: 0.0045
Epoch 133/200
 - 0s - loss: 

 - 0s - loss: 0.0030 - val_loss: 0.0033
Epoch 68/200
 - 0s - loss: 0.0030 - val_loss: 0.0033
Epoch 69/200
 - 0s - loss: 0.0030 - val_loss: 0.0033
Epoch 70/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 71/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 72/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 73/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 74/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 75/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 76/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 77/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 78/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 79/200
 - 0s - loss: 0.0029 - val_loss: 0.0033
Epoch 80/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 81/200
 - 0s - loss: 0.0029 - val_loss: 0.0032
Epoch 82/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 83/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 84/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 85/200
 - 0s - loss: 0.0028 - val_loss: 0.0032
Epoch 

 - 0s - loss: 0.0031 - val_loss: 0.0036
Epoch 20/200
 - 0s - loss: 0.0030 - val_loss: 0.0036
Epoch 21/200
 - 0s - loss: 0.0030 - val_loss: 0.0036
Epoch 22/200
 - 0s - loss: 0.0030 - val_loss: 0.0036
Epoch 23/200
 - 0s - loss: 0.0030 - val_loss: 0.0036
Epoch 24/200
 - 0s - loss: 0.0030 - val_loss: 0.0036
Epoch 25/200
 - 0s - loss: 0.0029 - val_loss: 0.0036
Epoch 26/200
 - 0s - loss: 0.0029 - val_loss: 0.0035
Epoch 27/200
 - 0s - loss: 0.0029 - val_loss: 0.0035
Epoch 28/200
 - 0s - loss: 0.0029 - val_loss: 0.0035
Epoch 29/200
 - 0s - loss: 0.0029 - val_loss: 0.0035
Epoch 30/200
 - 0s - loss: 0.0029 - val_loss: 0.0035
Epoch 31/200
 - 0s - loss: 0.0028 - val_loss: 0.0035
Epoch 32/200
 - 0s - loss: 0.0028 - val_loss: 0.0035
Epoch 33/200
 - 0s - loss: 0.0028 - val_loss: 0.0034
Epoch 34/200
 - 0s - loss: 0.0028 - val_loss: 0.0035
Epoch 35/200
 - 0s - loss: 0.0028 - val_loss: 0.0034
Epoch 36/200
 - 0s - loss: 0.0028 - val_loss: 0.0034
Epoch 37/200
 - 0s - loss: 0.0028 - val_loss: 0.0034
Epoch 

Epoch 173/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 174/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 175/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 176/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 177/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 178/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 179/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 180/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 181/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 182/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 183/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 184/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 185/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 186/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 187/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 188/200
 - 0s - loss: 0.0020 - val_loss: 0.0026
Epoch 189/200
 - 0s - loss: 0.0020 - val_loss: 0.0025
Epoch 190/200
 - 0s - loss: 0.0020 - val_loss: 0.0025
Epoch 191/200
 - 0s - loss: 

In [33]:
print(result10)

[[0.024610961671995772, 0.11900800458094891, 0.28768924213162334], [0.030422819913753757, 0.12770573618708397, 0.1533409831249188], [0.028318745502346112, 0.1257754124055262, 0.26643985607097226], [0.024146307830449635, 0.11785516819870284, 0.35888547583782193], [0.02874085728273705, 0.12776913983931287, 0.18812007505330613]]
